## Log Data Exporter - Combined Sensor CSV
This data exporter takes a log file and exports it to a single CSV files, with data values for each sensor.

Since each sensor is exported together, these CSV files will have interpolated data values.  This is because each row represents an update in _one_ sensor value, but there must be valid entries for the other sensors at the time. The values of all of the other sensors will be interpolated to the nearest valid reading (in time) for these events.

In [ ]:
import numpy as np
import pandas
import platypus.io.logs
import os

In [ ]:
# Import the data from the specified logfile
log_filename = '../logs/platypus_20161126_161922.txt'
data = platypus.io.logs.load(log_filename)

# To selectively export certain sensors, change this value to an array of names.
# By default, all sensors are exported (meaning they are also interpolated).
sensor_names = None
# sensor_names = ['ATLAS_DO', 'ES2']

In [ ]:
# Extract position of boat and remove duplicate entries.
data["pose"]["time"] = data["pose"].index
position = data['pose'][['time', 'latitude', 'longitude']].drop_duplicates(subset='time', keep='first')
position = position[['latitude', 'longitude']]  # remove unneeded time column after de-duplication

# If no sensor is specified, try to export all detected sensors.
if not sensor_names:
    sensor_names = [k for k in data.keys() if k not in ['pose', 'BATTERY']]

# Put together all the sensor data we are looking for.
# This also combines the name of the sensor with the name of each channel.    
sensor_frames = []
for sensor_name in sensor_names:
    sensor_data = data[sensor_name].copy()
    sensor_data.columns = [ sensor_name + '_' + str(k) for k in sensor_data.columns ]
    sensor_frames.append(sensor_data)
sensor_data = pandas.concat(sensor_frames, axis=1)

# Find the position for each sensor reading.
sensor_position = position.reindex(sensor_data.index, method='nearest')
output = pandas.concat((sensor_position, sensor_data), axis=1)

# Fill in missing values with last known values.
output = output.apply(pandas.Series.interpolate, method='nearest')
output['epoch_time'] = output.index.astype(np.int64)

In [ ]:
# Output the file as a CSV in the same directory as the original logfile.
output_filename = os.path.splitext(log_filename)[0] + '.csv'
output.to_csv(output_filename, index=True)